## Demonstrate word embedding using Gensim

In [1]:
import gensim
from nltk.corpus import brown

In [2]:
# nltk.download()

### Embeddings from scratch

In [10]:
train_set = brown.sents()[:10000]
train_set

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

Train word embeddings on brown corpus

In [12]:
model = gensim.models.Word2Vec(train_set)
model.save('brown.embedding')

In case it took long (not) save and reload model

In [3]:
loaded_model = gensim.models.Word2Vec.load('./models/brown.embedding')

In [4]:
# How many dims?

len(loaded_model.wv['practice'])

100

In [5]:
# cos sim example
print(loaded_model.wv.similarity('university','school'))
print(loaded_model.wv.similarity('university','home'))

0.99734426
0.9985765


### From pretrained model

In [6]:
word2vec_pretrained = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [7]:
len(word2vec_pretrained.vocab)

3000000

In [8]:
len(word2vec_pretrained['practice'])

300

In [9]:
print(word2vec_pretrained.wv.similarity('university','school'))
print(word2vec_pretrained.wv.similarity('university','home'))

0.5080745
0.09375003


C:\Users\esmba\AppData\Local\Temp\ipykernel_10348\3905516336.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  print(word2vec_pretrained.wv.similarity('university','school'))
C:\Users\esmba\AppData\Local\Temp\ipykernel_10348\3905516336.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  print(word2vec_pretrained.wv.similarity('university','home'))


In [10]:
word2vec_pretrained.most_similar(positive=['university'], topn = 3)

[('universities', 0.7003918886184692),
 ('faculty', 0.6780906915664673),
 ('unversity', 0.6758289933204651)]

In [11]:
word2vec_pretrained.doesnt_match('breakfast cereal dinner lunch'.split())

c:\Users\esmba\miniconda3\envs\CohereInterview\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'cereal'

In [12]:
word2vec_pretrained.most_similar(positive=['woman','king'], negative=['man'], topn = 1)

[('queen', 0.7118192911148071)]

In [13]:
word2vec_pretrained.most_similar(positive=['Paris','Germany'], negative=['Berlin'], topn = 1)

[('France', 0.7884093523025513)]

In [14]:

import numpy as np
labels = []
count = 0
max_count = 1000
X = np.zeros(shape=(max_count,len(word2vec_pretrained['university'])))

for term in word2vec_pretrained.index_to_key:
    X[count] = word2vec_pretrained[term]
    labels.append(term)
    count+= 1
    if count >= max_count: break

# It is recommended to use PCA first to reduce to ~50 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_50 = pca.fit_transform(X)

# Using TSNE to further reduce to 2 dimensions
from sklearn.manifold import TSNE
model_tsne = TSNE(n_components=2, random_state=0)
Y = model_tsne.fit_transform(X_50)

# Show the scatter plot
import matplotlib.pyplot as plt
plt.scatter(Y[:,0], Y[:,1], 20)

# Add labels
for label, x, y in zip(labels, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy = (x,y), xytext = (0, 0), textcoords = 'offset points', size = 10)

plt.show()


AttributeError: 'Word2VecKeyedVectors' object has no attribute 'index_to_key'